In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import time
import random

from functions.training import encoder_training, autoencoder_training
from functions.testing import encoder_testing, image_reconstruction
from functions.generation import latent_vector_statistics, image_creation, custom_image_creation
from functions.models import celeba_encoder_version_1, celeba_encoder_version_2, celeba_autoencoder_version_1, celeba_autoencoder_version_2, celeba_autoencoder_version_3, celeba_autoencoder_version_4

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# image transformation for 64x64
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

# loading CelebA dataset
training_data = datasets.CelebA(root = "./data", split = "train", download = True, transform = transform)
validation_data = datasets.CelebA(root = "./data", split = "valid", download = True, transform = transform)
testing_data = datasets.CelebA(root = "./data", split = "test", download = True, transform = transform)

# batch size for training and validation data
batch_size = 128

# data loaders for 64x64
training_loader = torch.utils.data.DataLoader(training_data, batch_size = batch_size, shuffle = True, 
                                              num_workers = 8, 
                                              pin_memory = True
                                             )
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size = batch_size, shuffle = True,
                                                num_workers = 8, 
                                                pin_memory = True
                                               )
testing_loader = torch.utils.data.DataLoader(testing_data, batch_size = 512, shuffle = False, 
                                             num_workers = 8, 
                                             pin_memory = True
                                            )

In [ ]:
# image transformation for 128x128
transform_2 = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor()])

# loading CelebA dataset
training_data_2 = datasets.CelebA(root = "./data", split = "train", download = True, transform = transform_2)
validation_data_2 = datasets.CelebA(root = "./data", split = "valid", download = True, transform = transform_2)
testing_data_2 = datasets.CelebA(root = "./data", split = "test", download = True, transform = transform_2)

# batch size for training and validation data
batch_size = 128

# data loaders for 128x128
training_loader_2 = torch.utils.data.DataLoader(training_data_2, batch_size = batch_size, shuffle = True, 
                                                num_workers = 8, 
                                                pin_memory = True
                                               )
validation_loader_2 = torch.utils.data.DataLoader(validation_data_2, batch_size = batch_size, shuffle = True, 
                                                  num_workers = 8, 
                                                  pin_memory = True
                                                 )
testing_loader_2 = torch.utils.data.DataLoader(testing_data_2, batch_size = 512, shuffle = False, 
                                               num_workers = 8, 
                                               pin_memory = True
                                              )

In [ ]:
# plotting images from the dataset 
fig, axes = plt.subplots(4, 15, figsize=(19, 5))

for j in range(15):
    axes[0, j].imshow(np.transpose(testing_data[j][0].numpy(), (1, 2, 0)))
    axes[1, j].imshow(np.transpose(testing_data[j + 15][0].numpy(), (1, 2, 0)))
    axes[2, j].imshow(np.transpose(testing_data_2[j][0].numpy(), (1, 2, 0)))
    axes[3, j].imshow(np.transpose(testing_data_2[j + 15][0].numpy(), (1, 2, 0)))

for i in range(4):
    for j in range(15):
        axes[i, j].axis("off")
    
plt.subplots_adjust(wspace = 0, hspace = 0)
plt.show()

In [ ]:
lr = 0.0005 # learning rate
nepoch = 30 # total epochs

encoder_model_1 = celeba_encoder_version_1().to(device) # loading model
criterion = nn.BCELoss() # loss function
optimiser = torch.optim.Adam(encoder_model_1.parameters(), lr)

start = time.time()

encoder_training(nepoch = nepoch, 
                 model = encoder_model_1, 
                 criterion = criterion, 
                 optimiser = optimiser, 
                 training_loader = training_loader,
                 validation_loader = validation_loader,
                 device = device, 
                 name = "celeba_encoder_version_1")

end = time.time()

print(f"training time: {(end - start)/60:.2f} minutes")
print(f"average training time per epoch: {((end - start)/60)/nepoch:.2f} minutes")

In [ ]:
# (epoch, training loss, validation loss)
celeba_encoder_version_1_graph = [(1, 0.4126950523, 0.3595162959), (2, 0.3240716426, 0.2967712877), (3, 0.2842113166, 0.2733318844), (4, 0.2695391095, 0.2609423843), (5, 0.2605512491, 0.2546784841), (6, 0.2535685669, 0.2515726715), (7, 0.2477877601, 0.2449690040), (8, 0.2436125028, 0.2417832752), (9, 0.2401496377, 0.2397705738), (10, 0.2374382203, 0.2360132099), (11, 0.2349338793, 0.2357215093), (12, 0.2329756030, 0.2349528678), (13, 0.2311931635, 0.2312704440), (14, 0.2294395963, 0.2326597898), (15, 0.2278663057, 0.2302323908), (16, 0.2262794499, 0.2289018565), (17, 0.2250184500, 0.2303040182), (18, 0.2236854610, 0.2294164785), (19, 0.2225954297, 0.2295120053), (20, 0.2213870878, 0.2281478159), (21, 0.2204211002, 0.2288863620), (22, 0.2190902992, 0.2271865329), (23, 0.2183000930, 0.2259961034), (24, 0.2173964739, 0.2256731229), (25, 0.2162483234, 0.2266265030), (26, 0.2156144113, 0.2273098608), (27, 0.2145258089, 0.2270234775), (28, 0.2138807864, 0.2264498682), (29, 0.2131324689, 0.2271751150), (30, 0.2123948170, 0.2265412112)]

In [ ]:
# loading model 
encoder_model_1 = celeba_encoder_version_1().to(device)
encoder_model_1.encoder.load_state_dict(torch.load("celeba_encoder_version_1.pth"))

In [ ]:
# label accuracy: 89.60%

encoder_testing(model = encoder_model_1, 
                testing_loader = testing_loader, 
                device = device)

In [ ]:
lr = 0.0003 # learning rate
nepoch = 17 # total epochs

encoder_model_2 = celeba_encoder_version_2().to(device) # loading model
criterion = nn.BCELoss() # loss function
optimiser = torch.optim.Adam(encoder_model_2.parameters(), lr)

start = time.time()

encoder_training(nepoch = nepoch, 
                 model = encoder_model_2, 
                 criterion = criterion, 
                 optimiser = optimiser, 
                 training_loader = training_loader_2,
                 validation_loader = validation_loader_2,
                 device = device, 
                 name = "celeba_encoder_version_2")

end = time.time()

print(f"training time: {(end - start)/60:.2f} minutes")
print(f"average training time per epoch: {((end - start)/60)/nepoch:.2f} minutes")

In [ ]:
# (epoch, training loss, validation loss)
celeba_encoder_version_2_graph = [(1, 0.4018851346, 0.3072722875), (2, 0.2880325653, 0.2714013847), (3, 0.2654104389, 0.2558503372), (4, 0.2524984536, 0.2454513872), (5, 0.2431889178, 0.2396684311), (6, 0.2363445004, 0.2340013913), (7, 0.2312514495, 0.2322115923), (8, 0.2271882823, 0.2268751314), (9, 0.2238632179, 0.2236403111), (10, 0.2209198048, 0.2222948160), (11, 0.2182511271, 0.2218674177), (12, 0.2156045550, 0.2209562589), (13, 0.2134033087, 0.2200322720), (14, 0.2110885754, 0.2190729127), (15, 0.2088379247, 0.2215062086), (16, 0.2069623555, 0.2178609672), (17, 0.2049612334, 0.2165376566), (18, 0.2030323673, 0.2172375786), (19, 0.2013226175, 0.2185391683), (20, 0.1993113114, 0.2188757804), (21, 0.1978028024, 0.2194687942), (22, 0.1959731221, 0.2199779389), (23, 0.1943999377, 0.2193507227), (24, 0.1926880595, 0.2178190940), (25, 0.1909114262, 0.2205138997), (26, 0.1893421746, 0.2224628879), (27, 0.1878021184, 0.2220258821), (28, 0.1860862701, 0.2244974266), (29, 0.1846252183, 0.2241531873), (30, 0.1829659542, 0.2254713263)]

In [ ]:
# loading model
encoder_model_2 = celeba_encoder_version_2().to(device)
encoder_model_2.encoder.load_state_dict(torch.load("celeba_encoder_version_2.pth"))

In [ ]:
# label accuracy: 89.98%

encoder_testing(model = encoder_model_2, 
                testing_loader = testing_loader_2, 
                device = device)

In [ ]:
graphs = [celeba_encoder_version_1_graph, celeba_encoder_version_2_graph]

# plotting encoder loss
fig, axes = plt.subplots(1, 2, figsize=(11, 2))

for i in range(2):
    encoder_version_epochs = [j[0] for j in graphs[i]]
    encoder_version_training_loss = [j[1] for j in graphs[i]]
    encoder_version_validation_loss = [j[2] for j in graphs[i]]

    axes[i].plot(encoder_version_epochs, encoder_version_training_loss, color = "r", linestyle = "--", label = "Training Loss")
    axes[i].plot(encoder_version_epochs, encoder_version_validation_loss, color = "b", alpha = 0.7, label = "Validation Loss")

    axes[i].set_xlabel("Epoch")
    axes[i].set_title(f"Encoder {i + 1}")
    axes[i].grid(True)
    axes[i].legend()

axes[0].set_ylabel("Loss")

plt.tight_layout()
plt.show()

In [ ]:
lr = 0.001
nepoch = 100

autoencoder_model_1 = celeba_autoencoder_version_1().to(device)
autoencoder_model_1.encoder.load_state_dict(torch.load("celeba_encoder_version_1.pth"))
criterion = nn.BCELoss()
optimiser = torch.optim.Adam(autoencoder_model_1.parameters(), lr)

start = time.time()

autoencoder_training(nepoch = nepoch, 
                     model = autoencoder_model_1, 
                     criterion = criterion, 
                     optimiser = optimiser, 
                     training_loader = training_loader,
                     validation_loader = validation_loader,
                     device = device, 
                     name = "celeba_autoencoder_version_1")

end = time.time()

print(f"training time: {(end - start)/60:.2f} minutes")
print(f"average training time per epoch: {((end - start)/60)/nepoch:.2f} minutes")

In [ ]:
# (epoch, training loss, validation loss)
celeba_autoencoder_version_1_graph = [(1, 0.5717694011, 0.5518544687), (2, 0.5449515314, 0.5414214692), (3, 0.5358514853, 0.5346723577), (4, 0.5315101549, 0.5302824103), (5, 0.5272486835, 0.5283735784), (6, 0.5255508191, 0.5265434132), (7, 0.5243768967, 0.5251764701), (8, 0.5230030799, 0.5242794588), (9, 0.5215345485, 0.5228556349), (10, 0.5210016754, 0.5228061898), (11, 0.5205999763, 0.5222892566), (12, 0.5198642495, 0.5217599101), (13, 0.5192713252, 0.5210299022), (14, 0.5187958289, 0.5206295985), (15, 0.5180908296, 0.5197866630), (16, 0.5176307795, 0.5232713734), (17, 0.5172463582, 0.5190820396), (18, 0.5169774011, 0.5184673715), (19, 0.5164996352, 0.5180629224), (20, 0.5161720135, 0.5179469098), (21, 0.5157037468, 0.5172665006), (22, 0.5153689330, 0.5173819726), (23, 0.5151946653, 0.5168239430), (24, 0.5150405145, 0.5176034006), (25, 0.5149215534, 0.5168736321), (26, 0.5148137706, 0.5175107914), (27, 0.5146459050, 0.5177876401), (28, 0.5142767343, 0.5162720512), (29, 0.5139044005, 0.5158585325), (30, 0.5137467396, 0.5160836713), (31, 0.5136043299, 0.5156787355), (32, 0.5134780695, 0.5152236156), (33, 0.5131987525, 0.5150856391), (34, 0.5128254158, 0.5146082175), (35, 0.5125954182, 0.5146243840), (36, 0.5124768703, 0.5144360310), (37, 0.5123918559, 0.5143061809), (38, 0.5123229205, 0.5145174238), (39, 0.5122180514, 0.5141422235), (40, 0.5121688373, 0.5146725587), (41, 0.5120490833, 0.5146049823), (42, 0.5118348229, 0.5141497761), (43, 0.5117098088, 0.5137064925), (44, 0.5116470054, 0.5137690711), (45, 0.5115688447, 0.5133307695), (46, 0.5115221258, 0.5138345231), (47, 0.5114787421, 0.5135579327), (48, 0.5114322845, 0.5135591070), (49, 0.5113854821, 0.5135505018), (50, 0.5113427029, 0.5136152114), 
                                      (51, 0.5112968765, 0.5143592375), (52, 0.5112497317, 0.5130787039), (53, 0.5112400399, 0.5134231815), (54, 0.5111623677, 0.5133363559), (55, 0.5110198788, 0.5127720676), (56, 0.5109160767, 0.5136476954), (57, 0.5108557015, 0.5128484047), (58, 0.5107633621, 0.5130341930), (59, 0.5106721438, 0.5128607759), (60, 0.5105327602, 0.5128969254), (61, 0.5104414360, 0.5125344510), (62, 0.5103671077, 0.5125497692), (63, 0.5103210592, 0.5127538928), (64, 0.5102681375, 0.5123533616), (65, 0.5102016765, 0.5128671135), (66, 0.5100210574, 0.5122517886), (67, 0.5099222664, 0.5118626215), (68, 0.5098690161, 0.5120776143), (69, 0.5098222507, 0.5117182134), (70, 0.5097785823, 0.5121794943), (71, 0.5097715163, 0.5119009155), (72, 0.5097283469, 0.5118580839), (73, 0.5096985124, 0.5121568153), (74, 0.5096626867, 0.5117867295), (75, 0.5096320374, 0.5120356156), (76, 0.5096269121, 0.5116438910), (77, 0.5095562266, 0.5123583227), (78, 0.5094813751, 0.5113897555), (79, 0.5093591301, 0.5115003588), (80, 0.5092994117, 0.5115235677), (81, 0.5092537916, 0.5113415332), (82, 0.5092099098, 0.5114668314), (83, 0.5091771274, 0.5112755417), (84, 0.5091571318, 0.5111642302), (85, 0.5091073463, 0.5119599589), (86, 0.5090576851, 0.5116412357), (87, 0.5090280836, 0.5118024013), (88, 0.5089647434, 0.5127853291), (89, 0.5089233907, 0.5109938242), (90, 0.5088257773, 0.5109203229), (91, 0.5088137615, 0.5109618262), (92, 0.5087700239, 0.5107270928), (93, 0.5087491978, 0.5112051981), (94, 0.5087069405, 0.5109120258), (95, 0.5086784307, 0.5109077505), (96, 0.5086400198, 0.5114463356), (97, 0.5086225651, 0.5106087123), (98, 0.5086025251, 0.5106525410), (99, 0.5085834727, 0.5109685973), (100, 0.5085462245, 0.5106570994)]

In [ ]:
# loading model
autoencoder_model_1 = celeba_autoencoder_version_1().to(device)
autoencoder_model_1.load_state_dict(torch.load("celeba_autoencoder_version_1.pth"))

In [ ]:
# reconstructing images
original_1, reconstructed_1 = image_reconstruction(model = autoencoder_model_1, testing_loader = testing_loader, device = device, count = 15)

In [ ]:
lr = 0.00065 # learning rate
nepoch = 100 # total epochs

autoencoder_model_2 = celeba_autoencoder_version_2().to(device) # loading model
criterion = nn.BCELoss() # loss function
optimiser = torch.optim.Adam(autoencoder_model_2.parameters(), lr)

start = time.time()

autoencoder_training(nepoch = nepoch, 
                     model = autoencoder_model_2, 
                     criterion = criterion, 
                     optimiser = optimiser, 
                     training_loader = training_loader,
                     validation_loader = validation_loader,
                     device = device, 
                     name = "celeba_autoencoder_version_2")

end = time.time()

print(f"training time: {(end - start)/60:.2f} minutes")
print(f"average training time per epoch: {((end - start)/60)/nepoch:.2f} minutes")

In [ ]:
# (epoch, training loss, validation loss)
celeba_autoencoder_version_2_graph = [(1, 0.5803902267, 0.5577472513), (2, 0.5478482718, 0.5452436946), (3, 0.5382329990, 0.5347970976), (4, 0.5317353364, 0.5330516211), (5, 0.5295645142, 0.5306270031), (6, 0.5263742654, 0.5263853971), (7, 0.5238329813, 0.5261474695), (8, 0.5223710715, 0.5242506315), (9, 0.5216931083, 0.5245693074), (10, 0.5208549194, 0.5226942087), (11, 0.5196313016, 0.5207501680), (12, 0.5183820249, 0.5207624199), (13, 0.5178951435, 0.5197542291), (14, 0.5171246842, 0.5188503384), (15, 0.5159745292, 0.5175422266), (16, 0.5152901111, 0.5173650539), (17, 0.5148891542, 0.5167564058), (18, 0.5143504009, 0.5164593275), (19, 0.5139963157, 0.5159154477), (20, 0.5134232365, 0.5157672316), (21, 0.5128880383, 0.5148094816), (22, 0.5126306355, 0.5149884505), (23, 0.5124581486, 0.5149640643), (24, 0.5122109134, 0.5142599820), (25, 0.5116689919, 0.5138448532), (26, 0.5111888259, 0.5134567886), (27, 0.5107833893, 0.5129305999), (28, 0.5104152478, 0.5126255500), (29, 0.5100025874, 0.5120990404), (30, 0.5096197721, 0.5118348404), (31, 0.5093478089, 0.5118833121), (32, 0.5090894276, 0.5112631285), (33, 0.5087644101, 0.5109270442), (34, 0.5084405732, 0.5112085772), (35, 0.5082277282, 0.5105251752), (36, 0.5081094550, 0.5109917642), (37, 0.5079648980, 0.5104432358), (38, 0.5078811668, 0.5104899082), (39, 0.5077000572, 0.5101291455), (40, 0.5074771543, 0.5097950063), (41, 0.5072582647, 0.5098989610), (42, 0.5071051466, 0.5096616993), (43, 0.5070219605, 0.5093688814), (44, 0.5069212995, 0.5095839464), (45, 0.5067576386, 0.5092260763), (46, 0.5065187972, 0.5089799063), (47, 0.5063641956, 0.5096039604), (48, 0.5062479738, 0.5091177009), (49, 0.5061657433, 0.5088983648), (50, 0.5060508381, 0.5085834311), 
                                      (51, 0.5058856172, 0.5088734543), (52, 0.5057694805, 0.5084829239), (53, 0.5056759420, 0.5084384053), (54, 0.5055546505, 0.5081222892), (55, 0.5053540262, 0.5078157211), (56, 0.5051615304, 0.5077561969), (57, 0.5050250880, 0.5083261524), (58, 0.5048814027, 0.5075455331), (59, 0.5047697780, 0.5071804401), (60, 0.5046448213, 0.5072019423), (61, 0.5045007134, 0.5071861003), (62, 0.5043998749, 0.5070473523), (63, 0.5043159727, 0.5069142519), (64, 0.5042260253, 0.5068978849), (65, 0.5041277204, 0.5068570184), (66, 0.5039944914, 0.5068421318), (67, 0.5038510249, 0.5063585958), (68, 0.5037392962, 0.5063977652), (69, 0.5036726206, 0.5064934355), (70, 0.5035902596, 0.5063701557), (71, 0.5035342208, 0.5062047151), (72, 0.5034746194, 0.5060845816), (73, 0.5033641255, 0.5061014553), (74, 0.5032523403, 0.5061760533), (75, 0.5031465679, 0.5059343920), (76, 0.5030637377, 0.5057965202), (77, 0.5029465787, 0.5054787182), (78, 0.5028532672, 0.5055353290), (79, 0.5027739408, 0.5053884788), (80, 0.5027140895, 0.5053745359), (81, 0.5026214091, 0.5050469394), (82, 0.5025841899, 0.5054677899), (83, 0.5025324466, 0.5054279530), (84, 0.5024763026, 0.5051104814), (85, 0.5024435354, 0.5054392503), (86, 0.5024182223, 0.5053448633), (87, 0.5023812695, 0.5050189539), (88, 0.5023340240, 0.5049883744), (89, 0.5023016279, 0.5050827618), (90, 0.5022459070, 0.5051573901), (91, 0.5021666298, 0.5049682934), (92, 0.5020962414, 0.5047076270), (93, 0.5020147627, 0.5050067166), (94, 0.5019031681, 0.5046814888), (95, 0.5018339810, 0.5046879214), (96, 0.5017586079, 0.5044978360), (97, 0.5016999851, 0.5042544254), (98, 0.5016473003, 0.5041907151), (99, 0.5016286932, 0.5044424412), (100, 0.5015739867, 0.5042807127)]

In [ ]:
# loading model
autoencoder_model_2 = celeba_autoencoder_version_2().to(device)
autoencoder_model_2.load_state_dict(torch.load("celeba_autoencoder_version_2.pth"))

In [ ]:
# reconstructing images
original_2, reconstructed_2 = image_reconstruction(model = autoencoder_model_2, testing_loader = testing_loader, device = device, count = 15)

In [ ]:
lr = 0.0005 # learning rate
nepoch = 10 # total epochs 

autoencoder_model_3 = celeba_autoencoder_version_3().to(device) # loading model
autoencoder_model_3.encoder.load_state_dict(torch.load("celeba_encoder_version_2.pth")) # loading pretrained paramaters 

for param in autoencoder_model_3.encoder.parameters(): 
    param.requires_grad = False # disabling encoder gradients calculation

criterion = nn.BCELoss() # loss function
optimiser = torch.optim.Adam(autoencoder_model_3.decoder.parameters(), lr)

start = time.time()

autoencoder_training(nepoch = nepoch, 
                     model = autoencoder_model_3, 
                     criterion = criterion, 
                     optimiser = optimiser, 
                     training_loader = training_loader_2,
                     validation_loader = validation_loader_2,
                     device = device, 
                     name = "celeba_autoencoder_version_3")

end = time.time()

print(f"training time: {(end - start)/60:.2f} minutes")
print(f"average training time per epoch: {((end - start)/60)/nepoch:.2f} minutes")

In [ ]:
# (epoch, training loss, validation loss)
celeba_autoencoder_version_3_graph = [(1, 0.6497155612, 0.6438789089), (2, 0.6413620434, 0.6407146408), (3, 0.6388874979, 0.6381946439), (4, 0.6370330863, 0.6376445932), (5, 0.6357120003, 0.6370846381), (6, 0.6345393836, 0.6370116434), (7, 0.6334774658, 0.6374869026), (8, 0.6323806364, 0.6361165750), (9, 0.6313171372, 0.6355622231), (10, 0.6303057947, 0.6356885601), (11, 0.6291591177, 0.6361062183), (12, 0.6279092881, 0.6360231875), (13, 0.6265120356, 0.6364954015), (14, 0.6251182411, 0.6372003743), (15, 0.6234953842, 0.6398916436), (16, 0.6217008541, 0.6379357958), (17, 0.6196955560, 0.6400615023), (18, 0.6173062618, 0.6407248638), (19, 0.6149051571, 0.6433041474), (20, 0.6122566361, 0.6445748779), (21, 0.6096198295, 0.6476637205), (22, 0.6068432982, 0.6452615318), (23, 0.6042213950, 0.6493015668), (24, 0.6014253441, 0.6519269683), (25, 0.5989773318, 0.6540594277)]

In [ ]:
# loading model 
autoencoder_model_3 = celeba_autoencoder_version_3().to(device)
autoencoder_model_3.load_state_dict(torch.load("celeba_autoencoder_version_3.pth"))

In [ ]:
# reconstructing images
original_3, reconstructed_3 = image_reconstruction(model = autoencoder_model_3, testing_loader = testing_loader_2, device = device, count = 15)

In [ ]:
lr = 0.0001 # learning rate
nepoch = 100 # total epochs

autoencoder_model_4 = celeba_autoencoder_version_4().to(device) # loading model
criterion = nn.MSELoss() # loss function
optimiser = torch.optim.Adam(autoencoder_model_4.parameters(), lr)

start = time.time()

autoencoder_training(nepoch = nepoch, 
                     model = autoencoder_model_4, 
                     criterion = criterion, 
                     optimiser = optimiser, 
                     training_loader = training_loader_2,
                     validation_loader = validation_loader_2,
                     device = device, 
                     name = "celeba_autoencoder_version_4")

end = time.time()

print(f"training time: {(end - start)/60:.2f} minutes")
print(f"average training time per epoch: {((end - start)/60)/nepoch:.2f} minutes")

In [ ]:
# (epoch, training loss, validation loss)
celeba_autoencoder_version_4_graph = [(1, 0.5273849468, 0.5139941197), (2, 0.5093045674, 0.5094143893), (3, 0.5058816264, 0.5094057918), (4, 0.5039165303, 0.5042471502), (5, 0.5024371599, 0.5034512975), (6, 0.5013578448, 0.5024056108), (7, 0.5004508377, 0.5017135772), (8, 0.4997507695, 0.5009643420), (9, 0.4990963782, 0.5000192510), (10, 0.4985583781, 0.4998274373), (11, 0.4980321201, 0.4993784487), (12, 0.4974855363, 0.4991396360), (13, 0.4969874885, 0.4992937677), (14, 0.4965557852, 0.4981245742), (15, 0.4961537672, 0.4971617399), (16, 0.4957557567, 0.4973081127), (17, 0.4954356045, 0.4967405834), (18, 0.4951932292, 0.4966100697), (19, 0.4949396327, 0.4966052948), (20, 0.4946971333, 0.4960570505), (21, 0.4945137419, 0.4965815397), (22, 0.4943011758, 0.4961430644), (23, 0.4941115031, 0.4962493603), (24, 0.4939256732, 0.4960830036), (25, 0.4937947110, 0.4957673219), (26, 0.4936274596, 0.4954448575), (27, 0.4935121019, 0.4950827991), (28, 0.4933785575, 0.4950252762), (29, 0.4932403610, 0.4948665288), (30, 0.4931381001, 0.4947250474), (31, 0.4929907056, 0.4946942364), (32, 0.4928938259, 0.4946549884), (33, 0.4928001706, 0.4946852332), (34, 0.4927337860, 0.4946436068), (35, 0.4926174626, 0.4950499718), (36, 0.4925515532, 0.4943038645), (37, 0.4924634889, 0.4945884114), (38, 0.4923491735, 0.4942780645), (39, 0.4922824136, 0.4942370464), (40, 0.4921972893, 0.4940877238), (41, 0.4921329972, 0.4942444771), (42, 0.4920505800, 0.4939598848), (43, 0.4920015582, 0.4938400875), (44, 0.4919286069, 0.4940352272), (45, 0.4918674105, 0.4937479286), (46, 0.4917988238, 0.4937786296), (47, 0.4917461551, 0.4938098798), (48, 0.4916826181, 0.4938107160), (49, 0.4916167632, 0.4936510217), (50, 0.4915703216, 0.4939663404), 
                                      (51, 0.4915358257, 0.4938715351), (52, 0.4914852115, 0.4934993165), (53, 0.4914387891, 0.4934977738), (54, 0.4913861169, 0.4939504252), (55, 0.4913631994, 0.4933330086), (56, 0.4913119635, 0.4936127567), (57, 0.4912846449, 0.4936668689), (58, 0.4912535792, 0.4937311617), (59, 0.4912081523, 0.4931141967), (60, 0.4911583450, 0.4933498177), (61, 0.4911238857, 0.4932195042), (62, 0.4910880212, 0.4931989973), (63, 0.4910596366, 0.4930150173), (64, 0.4910159440, 0.4931469504), (65, 0.4909842029, 0.4929662343), (66, 0.4909678886, 0.4931434169), (67, 0.4909278399, 0.4930195849), (68, 0.4908911235, 0.4931597981), (69, 0.4908440754, 0.4932490304), (70, 0.4908285831, 0.4929470821), (71, 0.4907998519, 0.4928976064), (72, 0.4907631597, 0.4928808858), (73, 0.4907263833, 0.4930151898), (74, 0.4907209063, 0.4929833372), (75, 0.4906766969, 0.4931211554), (76, 0.4906489456, 0.4927376675), (77, 0.4906289224, 0.4931314964), (78, 0.4906098567, 0.4930929096), (79, 0.4905852154, 0.4929148975), (80, 0.4905521592, 0.4927842472), (81, 0.4905478283, 0.4928854631), (82, 0.4905278258, 0.4930446467), (83, 0.4904877585, 0.4925503786), (84, 0.4904704610, 0.4929068111), (85, 0.4904508565, 0.4929604271), (86, 0.4904358048, 0.4926652083), (87, 0.4904108148, 0.4930173323), (88, 0.4903985906, 0.4928904548), (89, 0.4903717634, 0.4931877626), (90, 0.4903600093, 0.4928216279), (91, 0.4903269531, 0.4927742774), (92, 0.4903246945, 0.4928435756), (93, 0.4903001198, 0.4928063936), (94, 0.4902970871, 0.4926926757), (95, 0.4902833347, 0.4927674612), (96, 0.4902630492, 0.4926724747), (97, 0.4902430532, 0.4928440494), (98, 0.4902445112, 0.4927336092), (99, 0.4901984208, 0.4927191006), (100, 0.4901900091, 0.4927083521)]

In [ ]:
# loading model
autoencoder_model_4 = celeba_autoencoder_version_4().to(device)
autoencoder_model_4.load_state_dict(torch.load("celeba_autoencoder_version_4.pth"))

In [ ]:
# reconstructing images
original_4, reconstructed_4 = image_reconstruction(model = autoencoder_model_4, testing_loader = testing_loader_2, device = device, count = 15)

In [ ]:
graphs = [celeba_autoencoder_version_1_graph, celeba_autoencoder_version_2_graph, celeba_autoencoder_version_3_graph, 
          celeba_autoencoder_version_4_graph]

# plotting training/validation loss
fig, axes = plt.subplots(1, 4, figsize=(18, 2.5))

for i in range(4):
    autoencoder_version_epochs = [j[0] for j in graphs[i]]
    autoencoder_version_training_loss = [j[1] for j in graphs[i]]
    autoencoder_version_validation_loss = [j[2] for j in graphs[i]]

    axes[i].plot(autoencoder_version_epochs, autoencoder_version_training_loss, color = "r", linestyle = "--", label = "Training Loss")
    axes[i].plot(autoencoder_version_epochs, autoencoder_version_validation_loss, color = "b", alpha = 0.7, label = "Validation Loss")

    axes[i].set_xlabel("Epoch")
    axes[i].set_title(f"Autoencoder Version {i + 1}")
    axes[i].grid(True)
    axes[i].legend()

axes[0].set_ylabel("Loss")

plt.tight_layout()
plt.show()

In [ ]:
# plotting reconstructions
fig, axes = plt.subplots(5, 15, figsize=(15, 4))

for j in range(15):
    axes[0, j].imshow(np.transpose(original_1[j].numpy(), (1, 2, 0)))
    axes[1, j].imshow(np.transpose(reconstructed_1[j].numpy(), (1, 2, 0)))
    axes[2, j].imshow(np.transpose(reconstructed_2[j].numpy(), (1, 2, 0)))
    axes[3, j].imshow(np.transpose(reconstructed_3[j].numpy(), (1, 2, 0)))
    axes[4, j].imshow(np.transpose(reconstructed_4[j].numpy(), (1, 2, 0)))

for i in range(5):
    for j in range(15):
        axes[i, j].axis("off")

plt.subplots_adjust(wspace = 0.05, hspace = 0.05)
plt.show()

In [ ]:
autoencoder_model_1_mean, autoencoder_model_1_std = latent_vector_statistics(autoencoder_model_1, testing_loader, device)
autoencoder_model_2_mean, autoencoder_model_2_std = latent_vector_statistics(autoencoder_model_2, testing_loader, device)
autoencoder_model_3_mean, autoencoder_model_3_std = latent_vector_statistics(autoencoder_model_3, testing_loader_2, device)
autoencoder_model_4_mean, autoencoder_model_4_std = latent_vector_statistics(autoencoder_model_4, testing_loader_2, device)

print(f"autoencoder model 1: mean = {autoencoder_model_1_mean}, std = {autoencoder_model_1_std}")
print(f"autoencoder model 2: mean = {autoencoder_model_2_mean}, std = {autoencoder_model_2_std}")
print(f"autoencoder model 3: mean = {autoencoder_model_3_mean}, std = {autoencoder_model_3_std}")
print(f"autoencoder model 4: mean = {autoencoder_model_4_mean}, std = {autoencoder_model_4_std}")

In [ ]:
autoencoder_model_1_mean = torch.tensor([0.4912, 0.4870, 0.3125, 0.4607, 0.4753, 0.4331, 0.4703, 0.4521, 0.4723, 0.4813, 0.4847, 0.4979, 0.4612, 0.4628, 0.5137, 0.4541, 0.4704, 0.4763, 0.4665, 0.4702, 0.4883, 0.4791, 0.4195, 0.4801, 0.4820, 0.4740, 0.4605, 0.4713, 0.4697, 0.4311, 0.4572, 0.4777, 0.4252, 0.4705, 0.4580, 0.4638, 0.4593, 0.4228, 0.4540, 0.4615])
autoencoder_model_1_std = torch.tensor([0.0846, 0.0799, 0.1363, 0.0917, 0.0877, 0.1012, 0.0857, 0.1329, 0.0947, 0.1000, 0.1124, 0.0927, 0.0849, 0.0928, 0.1058, 0.0925, 0.0884, 0.0860, 0.0867, 0.0870, 0.0858, 0.0875, 0.1202, 0.0872, 0.0890, 0.0940, 0.0912, 0.0802, 0.0887, 0.1430, 0.0861, 0.0805, 0.1146, 0.1090, 0.0900, 0.0863, 0.0946, 0.1070, 0.0879, 0.0911])

In [ ]:
autoencoder_model_2_mean = torch.tensor([-0.0381, -0.0119,  0.1407, -0.0586,  0.1072,  0.0130,  0.7125,  0.5953, 0.0312, -0.0823, -0.0135, -0.0109,  0.1831,  0.0237, -0.0369,  0.0197, -0.0072,  0.0229,  0.0057, -0.0248,  0.2993, -0.1338,  0.0495,  0.5070, -0.0169, -0.0351, -0.0508, -0.0395,  0.8330, -0.0177, -0.0571,  0.0050, 0.0069, -0.0054, -0.0011, -0.5346,  0.0372, -0.0889, -0.2059,  0.0551, -0.0283,  0.0030,  0.6412, -0.0786,  0.2361, -0.0908,  0.7158, -0.0251, -0.0210,  0.5768, -0.3738, -0.0387,  0.0121, -0.1039, -0.0895, -0.0595, 0.1713, -0.0143, -0.0041, -0.3765, -0.0228, -0.0049,  0.0111, -0.0283, -0.0168, -0.0120, -0.0213, -0.0120, -0.0270,  0.0017,  0.0285, -0.0303, 0.7457,  0.0472, -0.1215, -0.0253,  0.0372, -0.0475, -0.0458, -0.3232, -0.0083,  0.0064,  0.0233, -0.0106])
autoencoder_model_2_std = torch.tensor([0.3733, 0.2632, 0.4775, 0.2608, 0.3223, 0.2385, 0.6485, 0.5676, 0.2940,0.4184, 0.2999, 0.2459, 0.3298, 0.2306, 0.2796, 0.4299, 0.3117, 0.2604,0.2741, 0.2505, 0.3991, 0.4443, 0.2849, 0.3518, 0.2880, 0.3351, 0.2718,0.3441, 0.5132, 0.3196, 0.2757, 0.2386, 0.2520, 0.2766, 0.2963, 0.7414,0.4208, 0.2535, 0.3921, 0.2183, 0.3072, 0.2174, 0.4823, 0.2528, 0.3106,0.3744, 0.4446, 0.2415, 0.2718, 0.8258, 0.6648, 0.2354, 0.2749, 0.3221,0.2722, 0.2537, 0.5340, 0.2183, 0.2832, 0.6733, 0.2932, 0.2567, 0.3065,0.2848, 0.2900, 0.2452, 0.2794, 0.3053, 0.2405, 0.3054, 0.2587, 0.2847,0.7536, 0.2608, 0.5377, 0.2911, 0.2581, 0.4069, 0.2802, 0.3428, 0.2567,0.2872, 0.3328, 0.2403])

In [ ]:
autoencoder_model_3_mean = torch.tensor([0.0952, 0.3094, 0.4644, 0.2154, 0.0184, 0.1671, 0.2464, 0.2434, 0.2848, 0.1288, 0.0710, 0.1709, 0.1169, 0.0701, 0.0556, 0.0599, 0.0703, 0.0397, 0.4005, 0.5171, 0.3815, 0.4569, 0.0406, 0.1381, 0.8435, 0.2596, 0.0274, 0.2765, 0.0900, 0.0705, 0.0526, 0.5008, 0.1900, 0.3090, 0.2289, 0.0441, 0.4964, 0.1272, 0.0680, 0.7645])
autoencoder_model_3_std = torch.tensor([0.1965, 0.3031, 0.3416, 0.2312, 0.0823, 0.3162, 0.1818, 0.2569, 0.3522, 0.2787, 0.1128, 0.2367, 0.2158, 0.1537, 0.1350, 0.2110, 0.1905, 0.1392, 0.4161, 0.3959, 0.4663, 0.4485, 0.1159, 0.1668, 0.3010, 0.1713, 0.0942, 0.2059, 0.1760, 0.1605, 0.1545, 0.4422, 0.1835, 0.3103, 0.2722, 0.1686, 0.4452, 0.1260, 0.1880, 0.2992])

In [ ]:
autoencoder_model_4_mean = torch.tensor([ 0.0052, -0.0184, -0.0122,  0.0352,  0.0276, -0.0897, -0.0387, -0.0207, 0.0558, -0.0133, -0.0135,  0.0429,  0.0611,  0.0534, -0.0741,  0.0669, -0.0388, -0.0202,  0.0200, -0.0441,  0.0172,  0.0013,  0.0311, -0.0163, -0.0612,  0.0039,  0.0725, -0.0278,  0.0527,  0.0161, -0.0289,  0.0015, -0.0191,  0.0376, -0.0241, -0.0216, -0.1233, -0.0023, -0.0728, -0.0441, -0.0113,  0.0408, -0.0549, -0.0673,  0.0132,  0.0458, -0.0008, -0.0444, 0.0429, -0.0696,  0.0607, -0.0442, -0.0150, -0.0118, -0.0347, -0.0420, -0.0020,  0.0227, -0.0415,  0.0253, -0.0577, -0.0224, -0.0519, -0.0391, 0.0908, -0.0071, -0.0388,  0.0781, -0.0272, -0.0111,  0.0124,  0.0200, 0.0648,  0.0185,  0.0659,  0.0059, -0.0431, -0.0171,  0.0035, -0.0873, -0.0191,  0.0002,  0.0659, -0.0629,  0.0307,  0.0156,  0.0290,  0.0404, -0.0590,  0.0261,  0.0226, -0.0225,  0.0698,  0.0684, -0.0481,  0.0131, -0.0697,  0.0938,  0.0200, -0.0849, -0.0223, -0.0125, -0.1023,  0.0095, -0.0364, -0.0953, -0.0327, -0.0357, -0.0058, -0.0047,  0.0198, -0.0542, -0.0508, -0.0196, -0.0070,  0.0931, -0.0931, -0.0266, -0.1024, -0.0680, -0.0589, -0.0028,  0.0038, -0.0490,  0.0957, -0.0110, -0.0256,  0.0473])
autoencoder_model_4_std = torch.tensor([0.9092, 0.9084, 0.9034, 1.3167, 0.8805, 0.9307, 0.9064, 0.9545, 0.9007, 0.8746, 0.8842, 1.2356, 0.9027, 1.3008, 0.8829, 1.6242, 0.9149, 1.4721, 0.8934, 0.8565, 0.8806, 0.9436, 0.8982, 0.9302, 0.8744, 0.9838, 0.9787, 0.8997, 0.8978, 0.9237, 0.8975, 0.9373, 1.3647, 0.9265, 0.8967, 0.9074, 0.9381, 0.8812, 0.9156, 1.0425, 0.9357, 0.8716, 1.6164, 0.8586, 0.9065, 0.9182, 0.9056, 1.2920, 0.8825, 1.0312, 0.9103, 0.9037, 0.8980, 0.9065, 0.9232, 0.9064, 0.9274, 0.9131, 0.8799, 0.9142, 0.8712, 0.9894, 0.9204, 0.9293, 0.9511, 0.9325, 0.9271, 0.8973, 1.2618, 0.8941, 0.9177, 1.3746, 0.9016, 1.0392, 0.8851, 0.9113, 1.4008, 0.9222, 0.9103, 0.9226, 0.9065, 0.9111, 1.2469, 0.9108, 1.4031, 0.9079, 0.9393, 0.9180, 0.8906, 0.9163, 0.9088, 0.8915, 0.9049, 0.9260, 0.9169, 0.9172, 0.8913, 0.8906, 0.9433, 0.8823, 0.9336, 0.8850, 0.8986, 0.9135, 1.0251, 0.9697, 0.9386, 0.9222, 0.9521, 0.9369, 0.8844, 1.4627, 0.9111, 0.9456, 0.9226, 0.9629, 0.9442, 1.3153, 0.9731, 0.9513, 0.9455, 0.9053, 1.1297, 0.8747, 0.8877, 0.9295, 0.9391, 0.8746])

In [ ]:
autoencoder_model_1_new_images = image_creation(model = autoencoder_model_1, device = device, mean = autoencoder_model_1_mean, 
                                                std = autoencoder_model_1_std, count = 12)
autoencoder_model_2_new_images = image_creation(model = autoencoder_model_2, device = device, mean = autoencoder_model_2_mean, 
                                                std = autoencoder_model_2_std, count = 12)
autoencoder_model_3_new_images = image_creation(model = autoencoder_model_3, device = device, mean = autoencoder_model_3_mean, 
                                                std = autoencoder_model_3_std, count = 12)
autoencoder_model_4_new_images = image_creation(model = autoencoder_model_4, device = device, mean = autoencoder_model_4_mean, 
                                                std = autoencoder_model_4_std, count = 12)

# plotting random created images
fig, axes = plt.subplots(4, 12, figsize = (15, 4))

for i in range(4):
    for j in range (12):
        axes[i, j].axis("off")

for j in range(12):
    axes[0, j].imshow(np.transpose(autoencoder_model_1_new_images[j].numpy(), (1, 2, 0)))
    axes[1, j].imshow(np.transpose(autoencoder_model_2_new_images[j].numpy(), (1, 2, 0)))
    axes[2, j].imshow(np.transpose(autoencoder_model_3_new_images[j].numpy(), (1, 2, 0)))
    axes[3, j].imshow(np.transpose(autoencoder_model_4_new_images[j].numpy(), (1, 2, 0)))

plt.subplots_adjust(wspace = 0.05, hspace = 0.05)
plt.show()

In [ ]:
celeba_attributes = {1: "5_o_clock_shadow", 
                     2: "arched_eyebrows", 
                     3: "attractive", 
                     4: "bags_under_eyes",
                     5: "bald",
                     6: "bangs",
                     7: "big_lips",
                     8: "big_nose",
                     9: "black_hair",
                     10: "blond_hair",
                     11: "blurry",
                     12: "brown_hair",
                     13: "bushy_eyebrows",
                     14: "chubby",
                     15: "double_chin",
                     16: "eyeglasses",
                     17: "goatee",
                     18: "gray_hair",
                     19: "heavy_makeup",
                     20: "high_cheekbones",
                     21: "male",
                     22: "mouth_slightly_open",
                     23: "mustache",
                     24: "narrow_eyes",
                     25: "no_beard",
                     26: "oval_face",
                     27: "pale_skin",
                     28: "pointy_nose",
                     29: "receding_hairline",
                     30: "rosy_cheeks",
                     31: "sideburns",
                     32: "smiling",
                     33: "straight_hair",
                     34: "wavy_hair",
                     35: "wearing_earrings",
                     36: "wearing_hat",
                     37: "wearing_lipstick",
                     38: "wearing_necklace",
                     39: "wearing_necktie",
                     40: "young"}

In [ ]:
                               # 1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
generic_person_1 = torch.tensor([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1])
generic_person_2 = torch.tensor([1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0])
generic_person_3 = torch.tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1])

fake_person_1 =    torch.tensor([0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1])
fake_person_2 =    torch.tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
fake_person_3 =    torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
data = [generic_person_1, generic_person_2, generic_person_3, fake_person_1, fake_person_2, fake_person_3]

custom_images_1 = custom_image_creation(model = autoencoder_model_1, device = device, data = data)
custom_images_3 = custom_image_creation(model = autoencoder_model_3, device = device, data = data)

# plotting custom made faces
fig, axes = plt.subplots(1, 12, figsize = (15, 1))

for j in range(6):
    axes[j].imshow(np.transpose(custom_images_1[j].numpy(), (1, 2, 0)))
    axes[j + 6].imshow(np.transpose(custom_images_3[j].numpy(), (1, 2, 0)))
    axes[j].axis("off")
    axes[j + 6].axis("off")

plt.subplots_adjust(wspace = 0.05, hspace = 0.05)
plt.show()